In [47]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

In [36]:
def limpiar(train):
    #train.insert(3, "mes", pd.to_datetime(train["fecha"]))
    #train["mes"] = train["mes"].dt.year
    train = train.drop(["titulo", "descripcion", "direccion", "ciudad", "lat", "lng", "fecha", "idzona", "id", "tipodepropiedad", 'provincia'], axis=1)
    
    for column in ["garages",'banos', 'habitaciones', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']:
        train[column].fillna(train[column].mode()[0], inplace=True)
        
    for column in ['antiguedad', 'metroscubiertos', 'metrostotales']:
        train[column].fillna(train[column].median(), inplace=True)
        
    #for colum in ["tipodepropiedad", "provincia"]:
    #    le = LabelEncoder()
    #    le.fit(train[colum])
    #    train[colum] = le.transform(train[colum])
        
    return train

In [21]:
def validar(X,y, metodo, transf=False):
    """
    X: datos de entrenamiento
    y: target de X
    metodo: Algoritmo a utilizar
    transf: Si aplicarle log al precio(normaliza el precio) antes de entrenar
    """
    # Se hace K-fold cross-validation
    kfold = KFold(5, True, 42)
    for i,(train_index, test_index) in enumerate(kfold.split(X)):
        # Se parte el dataset en dos, train y test, con el train entreno 
        # y con el test evaluo como predice mi algoritmo
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        if transf:
            y_train = np.log(y_train)
        # Me creo una instancia del modelo y luego lo entreno con los datos(con fit(X,y))
        reg_t = metodo()
        reg_t.fit(X_train,y_train)
        predicciones_t = reg_t.predict(X_test)
        if transf:
            predicciones_t = np.exp(predicciones_t)
        
        # Me fijo como me da el resultado
        print(f"Error de fold N°{i} es : {mean_absolute_error(y_test, predicciones_t)}")

In [22]:
def generar_submit(X,y, metodo,test, nombre, transf):
    """
    X: datos de entrenamiento
    y: target de X
    metodo: Algoritmo a utilizar
    test: Datos a predecir
    nombre: Nombre para el csv a generar
    transf: Si aplicarle log-transform a los datos antes de predecir
    """
    if transf:
        y = np.log(y)
    reg = metodo()
    reg.fit(X,y)
    
    ids = test["id"]
    test = limpiar(test)
    y_pred = reg.predict(test)
    if transf:
        y_pred = np.exp(y_pred)
        
    submit = pd.DataFrame(y_pred, columns=["target"])
    submit.insert(0, "id", ids)
    submit.to_csv(f"{nombre}.csv", index=False)

---

In [37]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
ejemplo = pd.read_csv("data/ejemploRespuesta.csv", names=["id", "precio"])

In [38]:
train = limpiar(train)

#### X es son los datos e y es el target(valor a predecir), en este caso el precio.

In [39]:
X, y = train.iloc[:,:-1],train.iloc[:,-1]

In [51]:
validar(X,y, LinearRegression, True)

Error de fold N°0 es : 1080009.1498675148
Error de fold N°1 es : 1076560.7563338603
Error de fold N°2 es : 1087944.502423703
Error de fold N°3 es : 1079722.2525840441
Error de fold N°4 es : 1077246.0832001395


In [52]:
generar_submit(X,y, LinearRegression, test, "submit_prueba", True)